In [22]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'

In [14]:
df = pd.read_csv("dataset/tweets.csv")
df

,Unnamed: 0.1,Unnamed: 0,Tweet Date,Tweet ID,Tweet Text,tweet_source,tweet_retweet_count,tweet_favorite_count,word_count,char_count,...,year,month,day,unique_word_count,stop_word_count,url_count,mean_word_length,hashtag_count,mention_count,burnout
0,0,0,2021-01-19,1351590096752017416,I'm a big music person. It speaks to me in all...,Twitter for Android,0,6,27,142,...,2021,1,19,25,15,1,4.296296,0,0,1
1,1,1,2021-01-19,1351555038863368193,@JohnWHarris15 I adore you and your unlimited ...,Twitter for Android,0,1,9,59,...,2021,1,19,9,4,0,5.555556,0,1,1
2,2,2,2021-01-19,1351554892926750720,@bubblydncer I have replied to countless texts...,Twitter for Android,0,1,22,142,...,2021,1,19,20,10,1,5.500000,0,1,1
3,3,3,2021-01-19,1351554395234824192,@MelBeer93 Ahhh I love this!,Twitter for Android,0,0,5,29,...,2021,1,19,5,1,0,4.800000,0,1,0
4,4,4,2021-01-19,1351554266247426050,@TheKimClub https://t.co/UUx3417HcD,Twitter for Android,0,1,2,35,...,2021,1,19,2,0,1,17.000000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874,1874,2026,2021-01-11,1348687433978798080,in my lecture rn talking about covid,Twitter for iPhone,0,0,7,37,...,2021,1,11,7,3,0,4.285714,0,0,1
1875,1875,2027,2021-01-11,1348687432938647552,@buzzycalz @skinnyqweeen @hearteyezshawty im i...,Twitter for iPhone,0,2,27,174,...,2021,1,11,26,11,0,5.444444,0,3,1
1876,1876,2028,2021-01-11,1348687432913473536,"@Death13__ Also this is just streams, videos a...",Twitter for Android,0,1,16,84,...,2021,1,11,16,8,0,4.312500,0,1,1
1877,1877,2029,2021-01-11,1348687432904892416,@Lyssss69 Honestly I just saw these on tiktok ...,Twitter for iPhone,0,1,15,74,...,2021,1,11,14,8,0,4.000000,0,1,0


In [15]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Tweet Date', 'Tweet ID', 'Tweet Text',
       'tweet_source', 'tweet_retweet_count', 'tweet_favorite_count',
       'word_count', 'char_count', 'word_density', 'punc_count', 'sentiment',
       'year', 'month', 'day', 'unique_word_count', 'stop_word_count',
       'url_count', 'mean_word_length', 'hashtag_count', 'mention_count',
       'burnout'],
      dtype='object')

In [16]:
waste_cols = ["Unnamed: 0.1","Unnamed: 0",'Tweet ID']
df = df.drop(waste_cols,axis=1)

In [17]:
df

,Tweet Date,Tweet Text,tweet_source,tweet_retweet_count,tweet_favorite_count,word_count,char_count,word_density,punc_count,sentiment,year,month,day,unique_word_count,stop_word_count,url_count,mean_word_length,hashtag_count,mention_count,burnout
0,2021-01-19,I'm a big music person. It speaks to me in all...,Twitter for Android,0,6,27,142,0.230769,12,-0.0625,2021,1,19,25,15,1,4.296296,0,0,1
1,2021-01-19,@JohnWHarris15 I adore you and your unlimited ...,Twitter for Android,0,1,9,59,0.176471,1,0.0000,2021,1,19,9,4,0,5.555556,0,1,1
2,2021-01-19,@bubblydncer I have replied to countless texts...,Twitter for Android,0,1,22,142,0.180328,14,-0.1250,2021,1,19,20,10,1,5.500000,0,1,1
3,2021-01-19,@MelBeer93 Ahhh I love this!,Twitter for Android,0,0,5,29,0.200000,2,0.6250,2021,1,19,5,1,0,4.800000,0,1,0
4,2021-01-19,@TheKimClub https://t.co/UUx3417HcD,Twitter for Android,0,1,2,35,0.057143,6,0.0000,2021,1,19,2,0,1,17.000000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874,2021-01-11,in my lecture rn talking about covid,Twitter for iPhone,0,0,7,37,0.225806,0,0.0000,2021,1,11,7,3,0,4.285714,0,0,1
1875,2021-01-11,@buzzycalz @skinnyqweeen @hearteyezshawty im i...,Twitter for iPhone,0,2,27,174,0.182432,8,-0.1000,2021,1,11,26,11,0,5.444444,0,3,1
1876,2021-01-11,"@Death13__ Also this is just streams, videos a...",Twitter for Android,0,1,16,84,0.228571,5,0.0000,2021,1,11,16,8,0,4.312500,0,1,1
1877,2021-01-11,@Lyssss69 Honestly I just saw these on tiktok ...,Twitter for iPhone,0,1,15,74,0.245902,2,0.6000,2021,1,11,14,8,0,4.000000,0,1,0


In [18]:
df.isnull().any()

Tweet Date              False
Tweet Text              False
tweet_source            False
tweet_retweet_count     False
tweet_favorite_count    False
word_count              False
char_count              False
word_density            False
punc_count              False
sentiment               False
year                    False
month                   False
day                     False
unique_word_count       False
stop_word_count         False
url_count               False
mean_word_length        False
hashtag_count           False
mention_count           False
burnout                 False
dtype: bool

In [20]:
df["tweet_retweet_count"].value_counts()

0      1484
1       114
2        54
3        35
4        29
       ... 
30        1
61        1
103       1
41        1
25        1
Name: tweet_retweet_count, Length: 67, dtype: int64

In [ ]:
# 